# 函数
函数是Python中最主要也是最重要的代码组织和复用手段。作为最重要的原则，如果你要重复使用相同或非常类似的代码，就需要写一个函数。通过给函数起一个名字，还可以提高代码的可读性。

函数使用def关键字声明，用return关键字返回值：

In [2]:
def my_function(x, y, z=1.5):
    if z > 1:
        return z * (x + y)
    else:
        return z / (x + y)

### 同时拥有多条return语句也是可以的。如果到达函数末尾时没有遇到任何一条return语句，则返回None。
函数可以有一些位置参数（positional）和一些关键字参数（keyword）。关键字参数通常用于指定默认值或可选参数。在上面的函数中，x和y是位置参数，而z则是关键字参数。也就是说，该函数可以下面这两种方式进行调用：


In [6]:
my_function(5, 6, z=0.7)
my_function(3.14, 7, 3.5)
my_function(10, 20)

45.0

### 函数参数的主要限制在于：关键字参数必须位于位置参数（如果有的话）之后。你可以任何顺序指定关键字参数。也就是说，你不用死记硬背函数参数的顺序，只要记得它们的名字就可以了。

# 命名空间、作用域，和局部函数
函数可以访问两种不同作用域中的变量：全局（global）和局部（local）。Python有一种更科学的用于描述变量作用域的名称，即命名空间（namespace）。任何在函数中赋值的变量默认都是被分配到局部命名空间（local namespace）中的。局部命名空间是在函数被调用时创建的，函数参数会立即填入该命名空间。在函数执行完毕之后，局部命名空间就会被销毁（会有一些例外的情况，具体请参见后面介绍闭包的那一节）。看看下面这个函数：

In [14]:
def func():
    a = []
    for i in range(5):
        a.append(i)

### 调用func()之后，首先会创建出空列表a，然后添加5个元素，最后a会在该函数退出的时候被销毁。假如我们像下面这样定义a：

In [3]:
a = []
def func():
    for i in range(5):
        a.append(i)

In [5]:
func()
print(a)

[0, 1, 2, 3, 4]


### 虽然可以在函数中对全局变量进行赋值操作，但是那些变量必须用global关键字声明成全局的才行：

In [1]:
a = None
def bind_a_variable():
    global a
    a = [1]

bind_a_variable()

In [2]:
print(a)

[1]


### 注意：我常常建议人们不要频繁使用global关键字。因为全局变量一般是用于存放系统的某些状态的。如果你发现自己用了很多，那可能就说明得要来点儿面向对象编程了（即使用类）。

# 返回多个值

In [15]:
def f():
    a = 5
    b = 6
    c = 7
    return a, b, c

In [16]:
a,b,c = f()

In [17]:
print(a,b,c)

5 6 7


### 该函数其实只返回了一个对象，也就是一个元组，最后该元组会被拆包到各个结果变量中,在上面的例子中，我们还可以这样写：return_value = f()

In [18]:
return_value = f()

In [19]:
print(return_value)

(5, 6, 7)


### 此外，还有一种非常具有吸引力的多值返回方式——返回字典：

In [20]:
def f():
    a = 5
    b = 6
    c = 7
    return {'a':a,'b':b,'c':c}

In [21]:
return_value = f()

In [22]:
print(return_value)

{'a': 5, 'b': 6, 'c': 7}


# 函数也是对象
由于Python函数都是对象，因此，在其他语言中较难表达的一些设计思想在Python中就要简单很多了。假设我们有下面这样一个字符串数组，希望对其进行一些数据清理工作并执行一堆转换：

In [23]:
states = ['   Alabama ', 'Georgia!', 'Georgia', 'georgia', 'FlOrIda', 'south   carolina##', 'West virginia?']

### 不管是谁，只要处理过由用户提交的调查数据，就能明白这种乱七八糟的数据是怎么一回事。为了得到一组能用于分析工作的格式统一的字符串，需要做很多事情：去除空白符、删除各种标点符号、正确的大写格式等。做法之一是使用内建的字符串方法和正则表达式re模块：

In [24]:
import re

def clean_strings(strings):
    result = []
    for value in strings:
        value = value.strip()
        value = re.sub('[!#?]', '', value)
        value = value.title()
        result.append(value)
    return result

In [25]:
clean_strings(states)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South   Carolina',
 'West Virginia']

### 其实还有另外一种不错的办法：将需要在一组给定字符串上执行的所有运算做成一个列表：

In [26]:
def remove_punctuation(value):
    return re.sub('[!#?]', '', value)

clean_ops = [str.strip, remove_punctuation, str.title]

def clean_strings(strings, ops):
    result = []
    for value in strings:
        for function in ops:
            value = function(value)
        result.append(value)
    return result

In [27]:
clean_strings(states, clean_ops)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South   Carolina',
 'West Virginia']

### 这种多函数模式使你能在很高的层次上轻松修改字符串的转换方式。此时的clean_strings也更具可复用性！

还可以将函数用作其他函数的参数，比如内置的map函数，它用于在一组数据上应用一个函数：

In [28]:
for x in map(remove_punctuation, states):
    print(x)

   Alabama 
Georgia
Georgia
georgia
FlOrIda
south   carolina
West virginia


# 匿名（lambda）函数
Python支持一种被称为匿名的、或lambda函数。它仅由单条语句组成，该语句的结果就是返回值。它是通过lambda关键字定义的，这个关键字没有别的含义，仅仅是说“我们正在声明的是一个匿名函数”。

In [33]:
def short_function(x):
    return x * 2

equiv_anon = lambda x: x * 2

### 本书其余部分一般将其称为lambda函数。它们在数据分析工作中非常方便，因为你会发现很多数据转换函数都以函数作为参数的。直接传入lambda函数比编写完整函数声明要少输入很多字（也更清晰），甚至比将lambda函数赋值给一个变量还要少输入很多字。看看下面这个简单得有些傻的例子：

In [11]:
def apply_to_list(some_list, f):
    return [f(x) for x in some_list]

ints = [4, 0, 1, 5, 6]
apply_to_list(ints, lambda x: x * 2)

[8, 0, 2, 10, 12]

### 虽然你可以直接编写[x *2for x in ints]，但是这里我们可以非常轻松地传入一个自定义运算给apply_to_list函数。

再来看另外一个例子。假设有一组字符串，你想要根据各字符串不同字母的数量对其进行排序：

In [16]:
strings = ['foo', 'card', 'bar', 'aaaa', 'abab']

In [17]:
strings.sort(key=lambda x: len(set(list(x))))

In [18]:
strings

['aaaa', 'foo', 'abab', 'bar', 'card']

In [27]:
x = 'foo'

In [28]:
list(x)

['f', 'o', 'o']

In [29]:
set(list(x))

{'f', 'o'}

In [30]:
len(set(list(x)))

2

### 笔记：lambda函数之所以会被称为匿名函数，与def声明的函数不同，原因之一就是这种函数对象本身是没有提供名称name属性。

# 柯里化：部分参数应用
柯里化（currying）是一个有趣的计算机科学术语，它指的是通过“部分参数应用”（partial argument application）从现有函数派生出新函数的技术。例如，假设我们有一个执行两数相加的简单函数：

In [39]:
def add_numbers(x, y):
    return x + y

### 通过这个函数，我们可以派生出一个新的只有一个参数的函数——add_five，它用于对其参数加5：

In [40]:
add_five = lambda y: add_numbers(5, y)

In [44]:
add_five(6)

11

### add_numbers的第二个参数称为“柯里化的”（curried）。这里没什么特别花哨的东西，因为我们其实就只是定义了一个可以调用现有函数的新函数而已。内置的functools模块可以用partial函数将此过程简化：

In [33]:
from functools import partial

In [34]:
add_five = partial(add_numbers, 5)

In [45]:
add_five(5)

10

# 生成器
能以一种一致的方式对序列进行迭代（比如列表中的对象或文件中的行）是Python的一个重要特点。这是通过一种叫做迭代器协议（iterator protocol，它是一种使对象可迭代的通用方式）的方式实现的，一个原生的使对象可迭代的方法。比如说，对字典进行迭代可以得到其所有的键：

In [46]:
some_dict = {'a': 1, 'b': 2, 'c': 3}

In [47]:
for key in some_dict:
    print(key)

a
b
c


### 当你编写for key in some_dict时，Python解释器首先会尝试从some_dict创建一个迭代器：

In [48]:
dict_iterator = iter(some_dict)

### 迭代器是一种特殊对象，它可以在诸如for循环之类的上下文中向Python解释器输送对象。大部分能接受列表之类的对象的方法也都可以接受任何可迭代对象。比如min、max、sum等内置方法以及list、tuple等类型构造器：

In [49]:
list(dict_iterator)

['a', 'b', 'c']

### 生成器（generator）是构造新的可迭代对象的一种简单方式。一般的函数执行之后只会返回单个值，而生成器则是以延迟的方式返回一个值序列，即每返回一个值之后暂停，直到下一个值被请求时再继续。要创建一个生成器，只需将函数中的return替换为yeild即可：

In [50]:
def squares(n=10):
    print('Generating squares from 1 to {0}'.format(n ** 2))
    for i in range(1, n + 1):
        yield i ** 2

### 调用该生成器时，没有任何代码会被立即执行：

In [69]:
gen = squares()

In [64]:
gen

<generator object squares at 0x0000000005D1A0A0>

### 直到你从该生成器中请求元素时，它才会开始执行其代码：

In [70]:
for x in gen:
    print(x, end= ' ')

Generating squares from 1 to 100
1 4 9 16 25 36 49 64 81 100 

# 生成器表达式
另一种更简洁的构造生成器的方法是使用生成器表达式（generator expression）。这是一种类似于列表、字典、集合推导式的生成器。其创建方式为，把列表推导式两端的方括号改成圆括号：

In [71]:
 gen = (x ** 2 for x in range(100))

### 它跟下面这个冗长得多的生成器是完全等价的：

In [72]:
def _make_gen():
    for x in range(100):
        yield x ** 2
gen = _make_gen()

### 生成器表达式也可以取代列表推导式，作为函数参数：

In [74]:
sum(x ** 2 for x in range(100))

328350

In [75]:
dict((i, i **2) for i in range(5))

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

# itertools模块
标准库itertools模块中有一组用于许多常见数据算法的生成器。例如，groupby可以接受任何序列和一个函数。它根据函数的返回值对序列中的连续元素进行分组。下面是一个例子：

In [76]:
import itertools

In [77]:
first_letter = lambda x: x[0]

In [78]:
names = ['Alan', 'Adam', 'Wes', 'Will', 'Albert', 'Steven']

In [79]:
for letter, names in itertools.groupby(names, first_letter):
    print(letter, list(names))

A ['Alan', 'Adam']
W ['Wes', 'Will']
A ['Albert']
S ['Steven']


# 错误和异常处理
优雅地处理Python的错误和异常是构建健壮程序的重要部分。在数据分析中，许多函数函数只用于部分输入。例如，Python的float函数可以将字符串转换成浮点数，但输入有误时，有ValueError错误：

In [80]:
float('1.234')

1.234

In [81]:
float('some')

ValueError: could not convert string to float: 'some'

### 假如想优雅地处理float的错误，让它返回输入值。我们可以写一个函数，在try/except中调用float：

In [82]:
def attempt_float(x):
    try:
        return float(x)
    except:
        return x

### 当float(x)抛出异常时，才会执行except的部分：

In [83]:
attempt_float('1.2345')

1.2345

In [84]:
attempt_float('some')

'some'

### 你可能注意到float抛出的异常不仅是ValueError：

In [85]:
float((1,2))

TypeError: float() argument must be a string or a number, not 'tuple'

### 你可能只想处理ValueError，TypeError错误（输入不是字符串或数值）可能是合理的bug。可以写一个异常类型：

In [86]:
def attempt_float(x):
    try:
        return float(x)
    except ValueError:
        return x

### 然后有：

In [87]:
attempt_float((1, 2))

TypeError: float() argument must be a string or a number, not 'tuple'

### 可以用元组包含多个异常：

In [88]:
def attempt_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return x

In [90]:
attempt_float((1,2))

(1, 2)

### 某些情况下，你可能不想抑制异常，你想无论try部分的代码是否成功，都执行一段代码。可以使用finally：

In [ ]:
f = open(path, 'w')

In [ ]:
try:
    write_to_file(f)
finally:
    f.close()

### 这里，文件处理f总会被关闭。相似的，你可以用else让只在try部分成功的情况下，才执行代码：

In [ ]:
f = open(path, 'w')

try:
    write_to_file(f)
except:
    print('Failed')
else:
    print('Succeeded')
finally:
    f.close()